## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key

from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [30]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")

# Remove unnecessary extra index column
new_order = ["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description"]
city_data_df = city_data_df[new_order]

# Formatting - this broke the code
city_data_df["Lat"] = city_data_df["Lat"].map("{:.2f}".format).astype(float)
city_data_df["Lng"] = city_data_df["Lng"].map("{:.2f}".format).astype(float)

city_data_df.head()

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Castro,BR,-24.79,-50.01,60.28,69,76,2.75,broken clouds
1,Hojo,JP,33.98,132.78,75.13,83,75,4.61,broken clouds
2,Havre-Saint-Pierre,CA,50.23,-63.60,61.52,94,90,11.18,overcast clouds
3,Rikitea,PF,-23.12,-134.97,72.18,76,100,9.66,overcast clouds
4,Avarua,CK,-21.21,-159.78,78.85,78,20,4.61,few clouds


In [31]:
city_data_df.dtypes
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature (°f) you would like for your trip? "))
max_temp = float(input("What is the maximum temperature (°f) you would like for your trip? "))

In [32]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]

In [33]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [40]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace("", np.nan)
clean_df = preferred_cities_df.dropna()

In [41]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hojo,JP,75.13,broken clouds,33.98,132.78,
4,Avarua,CK,78.85,few clouds,-21.21,-159.78,
6,Jamestown,US,84.06,clear sky,42.10,-79.24,
14,Carutapera,BR,76.89,broken clouds,-1.20,-46.02,
18,Hithadhoo,MV,81.68,overcast clouds,-0.60,73.08,
...,...,...,...,...,...,...,...
689,Sibolga,ID,75.07,overcast clouds,1.74,98.78,
692,Mineros,BO,78.64,clear sky,-17.12,-63.23,
700,Along,IN,75.97,overcast clouds,28.17,94.77,
702,Kutum,SD,81.28,clear sky,14.20,24.67,


In [36]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
    params["location"] = f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.") 

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [37]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'] = hotel_df['Hotel Name'].replace('', np.nan)
hotel_df.dropna()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hojo,JP,75.13,broken clouds,33.98,132.78,Seapa Makoto
4,Avarua,CK,78.85,few clouds,-21.21,-159.78,Paradise Inn
6,Jamestown,US,84.06,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
14,Carutapera,BR,76.89,broken clouds,-1.20,-46.02,Lidera
18,Hithadhoo,MV,81.68,overcast clouds,-0.60,73.08,Scoop Guest House
...,...,...,...,...,...,...,...
689,Sibolga,ID,75.07,overcast clouds,1.74,98.78,Hotel Syariah CN Darussalam
692,Mineros,BO,78.64,clear sky,-17.12,-63.23,Colpacaranda Eco Resort
700,Along,IN,75.97,overcast clouds,28.17,94.77,Hotel West
702,Kutum,SD,81.28,clear sky,14.20,24.67,Dibbo areaمنطقة دبو


In [38]:
# 8a. Create the output File (CSV)
clean_hotel_df = hotel_df
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [42]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Current Weather</b></dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))